In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [316]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\panag\AppData\Local\Temp/ipykernel_19148/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [317]:
page_url = 'https://www.novibet.gr/en/sports'
driver.get(page_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'acceptCookies_button'))).click()

In [318]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[data-cy="closeBtn"]').click()

## 1. Football

In [319]:
# Click Daily Coupon (Football)
dayly_coupon_button = driver.find_element(By.CSS_SELECTOR, 'a.ng-star-inserted[title="Daily coupon"]').click()

In [288]:
# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract footbal text
football_string = daily_coupon_body.text

In [310]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

football_list = [x for x in initial_list if x not in remove_elements]

### I want to find a way to increace the size of i if i insert elements

# Use padding for 'Markets are not available'
index = 0
while index < len(football_list):
    if football_list[index] == 'Markets are not available':
        football_list.insert(index+1, 'No_market')
        football_list.insert(index+2, 'No_market')
        football_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [311]:
# Create sublists based on Championship
championship = [x for x in football_list if ' - ' in x]
index_championship = [i for i,x in enumerate(football_list) if ' - ' in x]
sublists_championships = [football_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(football_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [313]:
# Create dataframe 
df_football = pd.DataFrame(columns = ['championship','team1','team2','time','1','odds_1','X','odds_X','2','odds_2',
                             'Over','odds_over','Under','odds_under','GG','odds_gg','NG','odds_ng'])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    team1_lst, team2_lst = [],[]
    time_lst = []
    one_lst, one_odds_lst, x_lst, x_odds_lst, two_lst, two_odds_lst = [],[],[],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    gg_lst, gg_odds_lst, ng_lst, ng_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 17)]
    

    i = 0
    while i < len(teams_only_lst):
        team1_lst.append(teams_only_lst[i])
        team2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        one_lst.append(teams_only_lst[i+3])
        one_odds_lst.append(teams_only_lst[i+4])
        x_lst.append(teams_only_lst[i+5])
        x_odds_lst.append(teams_only_lst[i+6])
        two_lst.append(teams_only_lst[i+7])
        two_odds_lst.append(teams_only_lst[i+8])
        over_lst.append(teams_only_lst[i+9])
        over_odds_lst.append(teams_only_lst[i+10])
        under_lst.append(teams_only_lst[i+11])
        under_odds_lst.append(teams_only_lst[i+12])
        gg_lst.append(teams_only_lst[i+13])
        gg_odds_lst.append(teams_only_lst[i+14])
        ng_lst.append(teams_only_lst[i+15])
        ng_odds_lst.append(teams_only_lst[i+16])
        i = i + 17

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'team1': team1_lst,
        'team2': team2_lst,
        'time': time_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        'X': x_lst,
        'odds_X': x_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        'GG': gg_lst,
        'odds_gg': gg_odds_lst,
        'NG': ng_lst,
        'odds_ng': ng_odds_lst
    })

    df_football = df_football.append(df, ignore_index = True)

In [314]:
df_football

,championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Africa - Africa Cup of Nations - Qualification,Benin,Rwanda,18:00,1,1.69,X,3.35,2,5.60,O 1.5,1.54,U 1.5,2.45,GG,2.45,NG,1.51
1,Africa - Africa Cup of Nations - Qualification,Sierra Leone,Sao Tome and Principe,19:00,1,1.11,X,8.00,2,16.00,O 3.5,1.89,U 3.5,1.85,Markets are not available,No_market,No_market,No_market
2,Africa - Africa Cup of Nations - Qualification,Blackburn Rovers (W),Charlton Athletic (W),20:45,1,3.70,X,3.85,2,1.76,O 2.5,1.93,U 2.5,1.78,GG,1.90,NG,1.81
3,Africa - Africa Cup of Nations - Qualification,Birmingham City (W),Crystal Palace (W),21:00,1,1.66,X,3.60,2,4.55,O 2.5,1.72,U 2.5,2.01,GG,1.71,NG,2.03
4,Africa - Africa Cup of Nations - Qualification,Madagascar,Central African Republic,Thu 15:00,1,1.86,X,3.40,2,4.20,O 2.5,2.30,U 2.5,1.60,GG,2.10,NG,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Brazil - Campeonato Brasileiro U20,Atletico Mineiro U20,Botafogo RJ U20,Thu 01:00,1,2.15,X,3.25,2,3.00,O 2.5,1.97,U 2.5,1.75,GG,1.79,NG,1.92
81,Africa - Africa Cup of Nations U23,Niger U23,Sudan U23,17:30,1,2.35,X,2.90,2,3.00,O 2.5,2.15,U 2.5,1.64,Markets are not available,No_market,No_market,No_market
82,Africa - Africa Cup of Nations U23,Egypt U23,Zambia U23,18:00,1,1.82,X,3.35,2,4.00,O 2.5,1.70,U 2.5,2.04,Markets are not available,No_market,No_market,No_market
83,Africa - Africa Cup of Nations U23,Senegal U23,Mali U23,21:00,1,2.04,X,3.05,2,3.55,O 2.5,2.35,U 2.5,1.53,Markets are not available,No_market,No_market,No_market


## 2. Basket

In [321]:
# Click the basketball button
basketball_button = driver.find_element(By.CSS_SELECTOR, '[class = "svgImage default BASKETBALL_GAME medium"]').click()

# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
daily_coupon_body = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract basketball text
basketball_string = daily_coupon_body.text

In [322]:
basketball_string

"Daily coupon\nFootball\nBasketball\nTennis\n24 hours\n12 hours\n3 hours\nPopular First\nUSA - NBA\nWAS Wizards\nDEN Nuggets\nThu 01:00\n+7.5\n1.97\n-7.5\n1.84\nO 226.5\n1.89\nU 226.5\n1.92\n1\n3.50\n2\n1.32\nDAL Mavericks\nGS Warriors\nThu 01:30\n+0.5\n1.93\n-0.5\n1.88\nO 235.5\n1.88\nU 235.5\n1.93\n1\n1.95\n2\n1.90\nMIA Heat\nNY Knicks\nThu 01:30\n-1.5\n1.89\n+1.5\n1.92\nO 224.5\n1.90\nU 224.5\n1.91\n1\n1.81\n2\n2.05\nTOR Raptors\nIND Pacers\nThu 01:30\n-8.5\n1.85\n+8.5\n1.97\nO 234.5\n1.90\nU 234.5\n1.91\n1\n1.28\n2\n3.85\nCHI Bulls\nPHI 76ers\nThu 02:00\n+3.5\n1.93\n-3.5\n1.88\nO 221.5\n1.94\nU 221.5\n1.87\n1\n2.45\n2\n1.59\nMEM Grizzlies\nHOU Rockets\nThu 02:00\n-12.5\n1.87\n+12.5\n1.94\nO 231.5\n1.91\nU 231.5\n1.90\n1\n1.14\n2\n6.30\nMIL Bucks\nSA Spurs\nThu 02:00\n-17.5\n1.91\n+17.5\n1.90\nO 237.5\n1.88\nU 237.5\n1.93\n1\n1.02\n2\n12.00\nMIN Timberwolves\nATL Hawks\nThu 02:00\n-4.5\n1.85\n+4.5\n1.96\nO 241.5\n1.93\nU 241.5\n1.88\n1\n1.51\n2\n2.65\nUTA Jazz\nPOR Blazers\nThu 03:0

In [337]:
# Create list from the initial string
initial_list = basketball_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

basketball_list = [x for x in initial_list if x not in remove_elements]

# Use padding for 'Markets are not available'
index = 0
while index < len(basketball_list):
    if basketball_list[index] == 'Markets are not available':
        basketball_list.insert(index+1, 'No_market')
        basketball_list.insert(index+2, 'No_market')
        basketball_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [338]:
basketball_list

['USA - NBA',
 'WAS Wizards',
 'DEN Nuggets',
 'Thu 01:00',
 '+7.5',
 '1.97',
 '-7.5',
 '1.84',
 'O 226.5',
 '1.89',
 'U 226.5',
 '1.92',
 '1',
 '3.50',
 '2',
 '1.32',
 'DAL Mavericks',
 'GS Warriors',
 'Thu 01:30',
 '+0.5',
 '1.93',
 '-0.5',
 '1.88',
 'O 235.5',
 '1.88',
 'U 235.5',
 '1.93',
 '1',
 '1.95',
 '2',
 '1.90',
 'MIA Heat',
 'NY Knicks',
 'Thu 01:30',
 '-1.5',
 '1.89',
 '+1.5',
 '1.92',
 'O 224.5',
 '1.90',
 'U 224.5',
 '1.91',
 '1',
 '1.81',
 '2',
 '2.05',
 'TOR Raptors',
 'IND Pacers',
 'Thu 01:30',
 '-8.5',
 '1.85',
 '+8.5',
 '1.97',
 'O 234.5',
 '1.90',
 'U 234.5',
 '1.91',
 '1',
 '1.28',
 '2',
 '3.85',
 'CHI Bulls',
 'PHI 76ers',
 'Thu 02:00',
 '+3.5',
 '1.93',
 '-3.5',
 '1.88',
 'O 221.5',
 '1.94',
 'U 221.5',
 '1.87',
 '1',
 '2.45',
 '2',
 '1.59',
 'MEM Grizzlies',
 'HOU Rockets',
 'Thu 02:00',
 '-12.5',
 '1.87',
 '+12.5',
 '1.94',
 'O 231.5',
 '1.91',
 'U 231.5',
 '1.90',
 '1',
 '1.14',
 '2',
 '6.30',
 'MIL Bucks',
 'SA Spurs',
 'Thu 02:00',
 '-17.5',
 '1.91',
 '+17.

In [335]:
# Create sublists based on Championship
championship = [x for x in basketball_list if ' - ' in x]
index_championship = [i for i,x in enumerate(basketball_list) if ' - ' in x]
sublists_championships = [basketball_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(basketball_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [336]:
sublists_championships[2]

['Europe - Eurocup (W)',
 'Umana Venezia (W)',
 'Galatasaray Istanbul (W)',
 'Thu 20:30',
 '+1.5',
 '1.86',
 '-1.5',
 '1.84',
 'O 138.5',
 '1.86',
 'U 138.5',
 '1.84']